# Projeto - Previsão de Risco de Crédito

## 1. Introdução

A análise de risco de crédito é um dos problemas mais clássicos e de maior impacto no setor financeiro. A concessão de crédito, seja para pessoas físicas ou jurídicas, é a base da atividade bancária, mas carrega consigo o risco inerente da inadimplência. Uma decisão de crédito equivocada pode levar a perdas financeiras significativas, enquanto uma política excessivamente conservadora pode resultar na perda de bons clientes e oportunidades de negócio valiosas.

Neste contexto, a ciência de dados emerge como uma ferramenta estratégica, permitindo que as instituições financeiras migrem de análises subjetivas e manuais para modelos preditivos objetivos, consistentes e escaláveis.

### 1.1. Descrição do problema

O problema central consiste em desenvolver um sistema inteligente capaz de classificar novos solicitantes de crédito em duas categorias: "bom risco" (provável que pague o empréstimo) e "mau risco" (provável que não pague o empréstimo).

Para esta análise, será utilizado conjunto de dados Statlog (German Credit Data), proveniente do repositório da UCI. Este dataset contém informações sobre 1.000 solicitantes de crédito na Alemanha, cada um descrito por 20 atributos, que incluem:

> Informações demográficas: idade, sexo, status civil.
> 
> Informações financeiras: status da conta corrente, histórico de crédito, valor e propósito do empréstimo, existência de poupança.
> 
> Informações profissionais e de moradia: tempo de emprego, tipo de trabalho, tipo de moradia.

A variável-alvo, risk, é binária, indicando se o cliente foi classificado como um bom ou mau pagador. Um desafio técnico relevante neste dataset é o desbalanceamento de classes, onde o número de "bons pagadores" é significativamente maior que o de "maus pagadores". Modelos treinados em dados desbalanceados podem se tornar tendenciosos, o que pode fazê-los performar mal ao identificar a classe minoritária, que é justamente a mais crítica para o negócio.

### 1.2. Objetivo

O objetivo principal deste projeto é construir um modelo de machine learning de classificação binária que preveja com alta acurácia e, mais importante, com alto poder de discriminação, a probabilidade de um solicitante de crédito se tornar inadimplente.

## 2. Configuração do Ambiente 

### 2.1 Importação de Bibliotecas

In [30]:
import pandas as pd
import numpy as np
import skrub
import logging

from src.data_ingestion import ingest_credit_data
from src import config

## 3. Carregamento dos Dados

### 3.1. Leitura do Conjunto de Dados

In [31]:
# Utiliza a URL definida no arquivo de configuração
url = config.GERMAN_CREDIT_ZIP_URL

# Carrega o arquivo através da URL
df = ingest_credit_data(url, True)

2025-10-22 16:17:23 - INFO - >>> INICIANDO O PIPELINE DE INGESTÃO DE DADOS <<<
2025-10-22 16:17:23 - INFO - Iniciando download de https://archive.ics.uci.edu/static/public/144/statlog+german+credit+data.zip
2025-10-22 16:17:24 - INFO - Arquivos extraídos com sucesso para: C:\Users\user\OneDrive\Documentos\HG\Projetos\Data Science\credit-risk-prediction\data\raw
2025-10-22 16:17:24 - INFO - Procurando por arquivo '.data' em 'C:\Users\user\OneDrive\Documentos\HG\Projetos\Data Science\credit-risk-prediction\data\raw'...
2025-10-22 16:17:24 - INFO - Arquivo alvo encontrado: 'german.data'
2025-10-22 16:17:24 - INFO - Arquivo renomeado para: 'credit_data.data'
2025-10-22 16:17:24 - INFO - Arquivo movido para: 'C:\Users\user\OneDrive\Documentos\HG\Projetos\Data Science\credit-risk-prediction\data\processed'
2025-10-22 16:17:24 - INFO - Arquivo extra 'german.data-numeric' removido.
2025-10-22 16:17:24 - INFO - Arquivo extra 'german.doc' removido.
2025-10-22 16:17:24 - INFO - Arquivo extra 'Ind

### 3.2. Visualização Inicial

In [32]:
 # Gera um relatório exploratório interativo da tabela utilizando o TableReport do pacote skrub, ignorando logs exibidos
logging.getLogger().setLevel(logging.ERROR)
skrub.TableReport(df).open()
logging.getLogger().setLevel(logging.INFO) 

Processing column  21 / 21


## 4. Limpeza dos Dados

### 4.1. Verificação de dados ausentes, nulos e duplicados

In [20]:
# Verifica o número de valores ausentes por coluna
missing_values = df.isnull().sum()
print("Valores ausentes por coluna:")
print(missing_values)

Valores ausentes por coluna:
checking_account_status       0
duration_in_month             0
credit_history                0
purpose                       0
credit_amount                 0
savings_account_bonds         0
present_employment_since      0
installment_rate_percent      0
personal_status_and_sex       0
other_debtors_guarantors      0
present_residence_since       0
property                      0
age_in_years                  0
other_installment_plans       0
housing                       0
number_of_existing_credits    0
job                           0
number_of_dependents          0
telephone                     0
foreign_worker                0
risk                          0
dtype: int64


In [24]:
# Conta o número total de valores NaN no DataFrame
num_nan = df.isnull().sum().sum()
print(f"Número total de valores NaN: {num_nan}")

# Conta o número total de valores nulos (incluindo None) no DataFrame
num_null = df.isna().sum().sum()
print(f"Número total de valores nulos: {num_null}")

Número total de valores NaN: 0
Número total de valores nulos: 0


In [25]:
# Verifica se existem valores duplicados
num_duplicates = df.duplicated().sum()
print(f"Número de linhas duplicadas: {num_duplicates}")

Número de linhas duplicadas: 0


In [27]:
# Exibe amostra de linhas duplicadas, caso exista
if num_duplicates > 0:
    print("Exemplo de linhas duplicadas:")
    display(df[df.duplicated()])